In [63]:
import random
import numpy as np
from scipy import sparse
from tools.luDecomposition import lu_decomposition
from tools.inverseMatrix import inverse_matrix
from tools.linearEquationsSystemSolve import linear_equations_system_solve
from tools.seidel import seidel
from tools.conditionalNumber import get_conditional_number

In [64]:
def generate_gilbert_matrix(k: int):
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = 1 / (i + j + 1)
    return matrix

In [65]:
def generate_diagonal_matrix(k):
    values = [0, -1, -2, -3, -4, -5, -6]
    noise = 10 ** (-k)
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = random.choice(values)
    for i in range(k):
        matrix[i][i] = -(sum(matrix[i]) - matrix[i][i]) + noise
    return matrix

In [66]:
def solutions(a: sparse.csr_matrix, b: np.array, short_output: bool = False):
    if not short_output:
        print("a =", a.todense())
        print("b =", b)
        print("Conditional number = ", get_conditional_number(a))
        print(">--------------------------------------------------------------<\n")

    answer_lu, iteration_count = linear_equations_system_solve(a, sparse.csr_matrix(b))
    print("Solution of system A with B vector (lu method)")
    if not short_output:
        print(answer_lu.transpose().toarray())

    print("Iteration number")
    print(iteration_count)
    print(">--------------------------------------------------------------<\n")

    answer_sei, iteration_count = seidel(a, b, 0.0001)
    print("Solution of system A with B vector (Seidel method)")
    if not short_output:
        print(answer_sei)
    print("Iteration number")
    print(iteration_count)
    print(">--------------------------------------------------------------<\n")

    print("Error for lu")
    print(np.linalg.norm(np.subtract(b, a.dot(answer_lu.transpose().toarray()[0]))))
    print(">--------------------------------------------------------------<\n")

    print("Error for Seidel")
    print(np.linalg.norm(np.subtract(b, a.dot(answer_sei))))
    print(">==============================================================<\n\n")

In [67]:
print("Random matrix 3x3")
solutions(sparse.csr_matrix([[10., -3., 5.],
                             [-7., 6., -1.],
                             [0., 2., 5.]]),
          np.array([1., 2., 3.]))

Random matrix 3x3
a = [[10. -3.  5.]
 [-7.  6. -1.]
 [ 0.  2.  5.]]
b = [1. 2. 3.]
Conditional number =  10.23482897453255
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[0.00689655 0.4137931  0.43448276]]
Iteration number
35
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[0.00698848 0.4138881  0.43444476]
Iteration number
90
>--------------------------------------------------------------<

Error for lu
2.220446049250313e-16
>--------------------------------------------------------------<

Error for Seidel
0.000445735182262944
>==============================================================<




In [68]:
print("Random matrix 4x4")
solutions(sparse.csr_matrix([[20., 0., 5., 4.],
                             [-7., 3., -1., 9],
                             [10., 2., -4., 2],
                             [2., 9., 11., 55.]])
              ,np.array([1., 2., 3., 4.]))


Random matrix 4x4
a = [[20.  0.  5.  4.]
 [-7.  3. -1.  9.]
 [10.  2. -4.  2.]
 [ 2.  9. 11. 55.]]
b = [1. 2. 3. 4.]
Conditional number =  86.60617988340239
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[ 7.69230769e-02  1.25000000e+00  7.93016446e-17 -1.34615385e-01]]
Iteration number
69
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[ 7.69279727e-02  1.24985097e+00 -3.87149027e-05 -1.34583433e-01]
Iteration number
352
>--------------------------------------------------------------<

Error for lu
2.7012892057857038e-15
>--------------------------------------------------------------<

Error for Seidel
0.00016125372834932436
>==============================================================<




In [69]:
print("Diagonal matrix 5x5 #1")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 5x5 #2")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 5x5 #3")
n = 5
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Diagonal matrix 10x10")
n = 10
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Diagonal matrix 50x50")
n = 50
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Diagonal matrix 100x100")
n = 100
a = sparse.csr_matrix(generate_diagonal_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

Diagonal matrix 5x5 #1
a = [[16.00001 -6.      -2.      -2.      -6.     ]
 [-3.      14.00001 -3.      -4.      -4.     ]
 [-6.      -2.      16.00001 -3.      -5.     ]
 [-5.      -1.      -6.      15.00001 -3.     ]
 [-2.      -2.      -6.       0.      10.00001]]
b = [-39.99999 -19.99998   1.00003  20.00004  26.00005]
Conditional number =  3889307.6976931565
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[1. 2. 3. 4. 5.]]
Iteration number
119
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[-3.0125295  -2.01252834 -1.01252723 -0.01252585  0.98747611]
Iteration number
175
>--------------------------------------------------------------<

Error for lu
9.39959687352353e-15
>--------------------------------------------------------------<

Error for Seidel
0.0001244038041889491
>==============================================================<


Diagonal m

In [70]:
print("Gilbert matrix 5x5")
n = 5
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b)

print("Gilbert matrix 10x10")
n = 10
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)

print("Gilbert matrix 50x50")
n = 50
a = sparse.csr_matrix(generate_gilbert_matrix(n))
ans = np.array([i for i in range(1, n + 1)])
b = a.dot(ans)
solutions(a, b, True)


Gilbert matrix 5x5
a = [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
b = [5.         3.55       2.81428571 2.34642857 2.01746032]
Conditional number =  480849.1169944144
>--------------------------------------------------------------<

Solution of system A with B vector (lu method)
[[1. 2. 3. 4. 5.]]
Iteration number
119
>--------------------------------------------------------------<

Solution of system A with B vector (Seidel method)
[0.99999625 2.02639721 2.82444736 4.32423602 4.82135593]
Iteration number
34725
>--------------------------------------------------------------<

Error for lu
0.0
>--------------------------------------------------------------<

Error for Seidel
2.205593874610787e-05
>==============================================